# Tarea 07 - Chatbot RAG

***Inteligencia Artificial***

***II Semestre 2024***

***Tecnológico de Costa Rica***

Estudiantes: 
- Esteban Guzmán
- Rolando Mora

In [ ]:
import os
import re
import time
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec

INDEX_NAME = "chatbotrag"
NAMESPACE = f"{INDEX_NAME}-namespace"

## Parte 1: Preparación del Corpus y Cálculo de Embeddings

### 1. Elección del Dataset

Se escogió el dataset de `BBC Full Text Document Classification` del sitio web de [Kaggle](https://www.kaggle.com/datasets/shivamkushwaha/bbc-full-text-document-classification).

Se usarán los datos referentes a tecnología.

In [59]:
# Extraemos el contenido de los artículos
def extract_articles(folder_path: str, max_articles) -> list[str]:
	articles = []
	for filename in os.listdir(folder_path)[:max_articles]:
		with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
			content = file.read()
			articles.append(content)
	return articles

### 2. Preprocesamiento

In [69]:
# Limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^a-zA-Záéíóúñü\s.]', '', text)  # Eliminar caracteres especiales, manteniendo puntos
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios extra
    return text

# Segmentar el texto en párrafos de 4 oraciones
def segment_text(text, doable=True):
    if not doable: return [text]
    sentences_per_fragment = 4
    # Dividir el texto en oraciones
    sentences = text.split(".")
    start = 0
    fragments = []
    # Separa el texto en la cantidad de oraciones
    while start < len(sentences):
        end = start + sentences_per_fragment
        fragments.append(".".join(sentences[start:end]))
        start = end
    return fragments

### 3. Generación de Embeddings

In [51]:
def index_articles(articles):
	plain_articles = []
	for i, article in enumerate(articles):
		for j, paragraph in enumerate(article):
			plain_articles.append({"id": f"{i},{j}", "text": paragraph})
	return plain_articles

def generate_embeddings(pc, indexed_articles):
	embeddings = pc.inference.embed(
		model="multilingual-e5-large",
		inputs=[a["text"] for a in indexed_articles],
		parameters={"input_type": "passage", "truncate": "END"}
	)

	return embeddings

## Parte 2: Configuración de la Base de Datos Vectorial

### 1. Elección de la base de datos vectorial
Se seleccionó Pinecone debido a su almacenamiento en la nube y al no ser necesario más instalaciones más que la librería de python.

In [52]:
def get_pinecone():
    return PineconeGRPC(api_key="81002466-39d5-4599-a684-9b08a1c70aee")

def delete_index(pc):
    pc.delete_index(INDEX_NAME)

### 2. Carga de datos

In [53]:
def save_embeddings(pc, indexed_articles, embeddings):
	# Creación del índice de pinecone
	if not pc.has_index(INDEX_NAME):
		pc.create_index(
			name=INDEX_NAME,
			dimension=1024,
			metric="cosine",
			spec=ServerlessSpec(
				cloud="aws", 
				region="us-east-1"
			) 
		) 

	# Se espera hasta que el índice esté listo
	while not pc.describe_index(INDEX_NAME).status["ready"]:
		time.sleep(1)

	index = pc.Index(INDEX_NAME)

	# Se preparan los registros
	records = []
	for d, e in zip(indexed_articles, embeddings):
		records.append({
			"id": d["id"],
			"values": e["values"],
			"metadata": {"text": d["text"]}
		})

	index.upsert(
		vectors=records,
		namespace=NAMESPACE
	)

	# Se espera la indexación de los vectores
	time.sleep(10)

	return index

## Parte 3: Diseño de Chatbot con RAG

### 1. Desarrollo del flujo de recuperación

In [54]:
def answer_query(pc, index, query):
	# Se hace embedding al query
	query_embedding = pc.inference.embed(
		model="multilingual-e5-large",
		inputs=[query],
		parameters={
			"input_type": "query"
		}
	)

	# Se realiza la consulta a la base de datos
	results = index.query(
		namespace=NAMESPACE,
		vector=query_embedding[0].values,
		top_k=3,
		include_values=False,
		include_metadata=True
	)

	return results

---

In [73]:
# Se limita a los primeros 95 artículos por limitaciones de la base de datos
articles = extract_articles("./dataset/bbc-fulltext (document classification)/bbc/tech", 95)
articles = list(map(clean_text, articles))
# No se segmentan los artículos para no pasar la limitación en la base de datos
articles = list(map(lambda article: segment_text(article, False), articles))

pc = get_pinecone()
indexed_articles = index_articles(articles)

embeddings = generate_embeddings(pc, indexed_articles)
index = save_embeddings(pc, indexed_articles, embeddings)

query = input("Consulta: ")
while query:
    print("QUERY: ", query)
    answer = answer_query(pc, index, query)
    print(answer)
    query = input("Consulta: ")
    
delete_index(pc)

QUERY:  Which console hits US in March?
{'matches': [{'id': '24,0',
              'metadata': {'text': 'sony psp console hits us in march us '
                                   'gamers will be able to buy sonys '
                                   'playstation portable from march but there '
                                   'is no news of a europe debut. the handheld '
                                   'console will go on sale for and the first '
                                   'million sold will come with spiderman on '
                                   'umd the disc format for the machine. sony '
                                   'has billed the machine as the walkman of '
                                   'the st century and has sold more than '
                                   'units in japan. the console cm by .cm will '
                                   'play games movies and music and also '
                                   'offers support for wireless gaming. son